In [1]:
import sys
sys.path.insert(0, './PythonFiles/')

import pandas as pd
from sklearn.preprocessing import LabelEncoder
import ast
import unicodedata
import math
import warnings
import pickle
from CategoryChanger import CategoryChanger
from Classifiers import Classifiers
from DFPreProcessing import CleanDF, DataPreProcessor
from sklearn.metrics import *
from sklearn import model_selection

In [11]:
import random

def random_prediction(test):
    prediction = []
    for i in range(len(test)):
        num = random.randint(0,1)
        if num == 0:
            prediction.append('NScam')
        else:
            prediction.append('Scam')
    return prediction

def all_scores(Test_Y, prediction):
    labels = ["NScam","Scam"]
    scores = {"precision":{"NScam":0,"Scam":0},"recall":{"NScam":0,"Scam":0},"f1-score":{"NScam":0,"Scam":0},"accuracy":0}

    for label in labels:
        scores["precision"][label] = precision_score(Test_Y, prediction, pos_label=label)
        scores["recall"][label] = recall_score(Test_Y, prediction, pos_label=label)
        scores["f1-score"][label] = f1_score(Test_Y, prediction, pos_label=label)
    
    scores["accuracy"] = accuracy_score(Test_Y, prediction)

    return scores

def get_avg(scores):
    avg_scores = scores[0]

    for score in scores[1:]:
        for key, value in score.items():
            if isinstance(value, dict):
                for innerKey, innerValue in value.items():
                    avg_scores[key][innerKey] += innerValue
            else:
                avg_scores[key] += value

    for key, value in avg_scores.items():
        if isinstance(value, dict):
            for innerKey, innerValue in value.items():
                avg_scores[key][innerKey] = round(innerValue/len(scores), 2)
        else:
            avg_scores[key] = round(value/len(scores), 2)
        
    return avg_scores


In [12]:
def find_value_state(val):
    if int(val) > 2017:
        return "Scam"
    elif int(val) < 2018:
        return "NScam"
    else:
        return "Unknown"

In [4]:
features_to_be_classified = [['text_final','Location','Image_Counts']]

trainData = pd.read_csv('trainData.csv', low_memory=False)
testData = pd.read_csv('testData.csv', low_memory=False)

trainData = CleanDF(trainData).getConvertedDF()

In [5]:
for indx,row in enumerate(trainData['Join_date']):
    trainData.loc[indx,'label'] = str(find_value_state(trainData['Join_date'][indx]))

In [6]:
test = DataPreProcessor().processData(testData.copy(deep=True))
Test_X = DataPreProcessor().get_feature_datasets(test, features_to_be_classified)[0]
Test_Y = testData['label']

In [ ]:
classif = Classifiers()


In [9]:
loop_cnt = 1000
scores = []

for i in range(loop_cnt):
    prediction = random_prediction(Test_X)
    scores.append(all_scores(Test_Y,prediction))

avg_scores = get_avg(scores)

In [10]:
avg_scores

{'precision': {'NScam': 0.52, 'Scam': 0.48},
 'recall': {'NScam': 0.5, 'Scam': 0.5},
 'f1-score': {'NScam': 0.51, 'Scam': 0.49},
 'accuracy': 0.5}